In [1]:
# Imports

import os
from tkinter import *
from tkinter import filedialog, simpledialog
from PIL import ImageTk, Image
import numpy as np

from skimage.color import rgb2gray
from skimage.filters import hessian, laplace, meijering, roberts, sato, sobel, unsharp_mask, threshold_li, threshold_otsu, threshold_yen, gaussian

import matplotlib.pyplot as plt
from matplotlib.pyplot import hist, title
from skimage.exposure import match_histograms, rescale_intensity

from skimage.transform import resize, rotate, rescale, swirl, integral_image
from skimage.transform import SimilarityTransform

from skimage.morphology import convex_hull_image, area_closing, binary_closing, black_tophat, dilation, erosion, local_maxima, local_minima, watershed, white_tophat

from skimage.segmentation import active_contour

import cv2
from skimage import io

In [2]:
# File

filePath = "Samples/Blank.png"

def fitImageToCanvas(image):
    w, h = image.size
    if w > h: return image.resize((400, int(400 * h / w)), Image.ANTIALIAS)
    elif h > w: return image.resize((int(400 * w / h), 400), Image.ANTIALIAS)
    else: return image.resize((400, 400), Image.ANTIALIAS)

def openImageFileDialog():
    global filePath
    filePath = filedialog.askopenfilename(title = "Select an image",
                filetypes=[("Image Files", (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"))])
    
    global defaultImage
    defaultImage = Image.open(filePath)
    defaultImage = fitImageToCanvas(defaultImage)
    defaultImage = ImageTk.PhotoImage(defaultImage)
    defaultPanel.configure(image = defaultImage)
    defaultPanel.image = defaultImage
    
    global processedImageFirst
    processedImageFirst = Image.open(filePath)
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    processedPanel.configure(image = defaultImage)
    processedPanel.image = processedImage
    
    listbox.delete(0, END)
    
def saveImage():
    filename = filedialog.asksaveasfile(mode="w", defaultextension=".png", filetypes=[("Image Files", (".png"))])
    if not filename: return
    global processedPanel
    processedPanel.image._PhotoImage__photo.write(filename.name)

In [3]:
# Filters

def filtersOther(kind):
    global processedImageFirst
    image = rgb2gray(np.asarray(processedImageFirst))
    
    if kind=="hessian":
        filtered = hessian(image)
        listbox.insert(listbox.size() + 1, "hessian") 
    if kind=="laplace":
        filtered = laplace(image)
        listbox.insert(listbox.size() + 1, "laplace")
    if kind=="meijering":
        filtered = meijering(image)
        listbox.insert(listbox.size() + 1, "meijering")
    if kind=="roberts":
        filtered = roberts(image)
        listbox.insert(listbox.size() + 1, "roberts")
    if kind=="sato":
        filtered = sato(image)
        listbox.insert(listbox.size() + 1, "sato")
    if kind=="sobel":
        filtered = sobel(image)
        listbox.insert(listbox.size() + 1, "sobel")
    if kind=="unsharp_mask":
        filtered = unsharp_mask(image)
        listbox.insert(listbox.size() + 1, "unsharp_mask")
    
    processedImageFirst = Image.fromarray(filtered * 255)
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

def filtersThreshold(kind):
    global processedImageFirst
    image = rgb2gray(np.asarray(processedImageFirst))
    
    if kind=="li":
        thresh = threshold_li(image)
        listbox.insert(listbox.size() + 1, "threshold_li")
    if kind=="otsu":
        thresh = threshold_otsu(image)
        listbox.insert(listbox.size() + 1, "threshold_otsu")
    if kind=="yen":
        thresh = threshold_yen(image)
        listbox.insert(listbox.size() + 1, "threshold_yen")
    
    binary = image > thresh
    
    processedImageFirst = Image.fromarray(binary)
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

In [4]:
# Histogram

def histogramShowing():
    global processedImageFirst
    image = np.asarray(processedImageFirst)
    
    %matplotlib qt
    hist(image.flatten(), 256, range=(0,256))
    title("Histogram")
    
def histogramMatching():
    global processedImageFirst
    image = np.asarray(processedImageFirst.convert("RGB"))
    
    reference = Image.open(filedialog.askopenfilename(title = "open",
                filetypes=[("Image Files", (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"))]))
    reference = np.asarray(reference.convert("RGB"))
    listbox.insert(listbox.size() + 1, "histogram_matching")
    
    matched = match_histograms(image, reference, multichannel=True)
    
    processedImageFirst = Image.fromarray(matched)
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

In [5]:
# Transformations

def transformations(kind):
    global processedImageFirst
    image = np.asarray(processedImageFirst)
    
    if kind == "resize":
        newWidth = simpledialog.askstring("Image Width", "Width", parent=root)
        newHeight = simpledialog.askstring("Image Height", "Height", parent=root)
        transformedImage = resize(image, (int(newWidth), int(newHeight)), anti_aliasing=True)
        listbox.insert(listbox.size() + 1, "resize")
    if kind == "rotate":
        value = simpledialog.askstring("Rotation", "Value", parent=root)
        transformedImage = rotate(image, int(value), resize=True)
        listbox.insert(listbox.size() + 1, "rotate")
    if kind == "rescale":
        value = simpledialog.askstring("Rescale", "Value", parent=root)
        transformedImage = rescale(image, float(value), anti_aliasing=False, multichannel=True)
        listbox.insert(listbox.size() + 1, "rescale")
    if kind == "swirl":
        rotationValue = simpledialog.askstring("Rotation", "Value", parent=root)
        strengthValue = simpledialog.askstring("Strength", "Value", parent=root)
        radiusValue = simpledialog.askstring("Radius", "Value", parent=root)
        transformedImage = swirl(image, rotation=int(rotationValue), strength=int(strengthValue), radius=int(radiusValue))
        listbox.insert(listbox.size() + 1, "swirl")
    if kind == "integral_image":
        transformedImage = integral_image(image)
        listbox.insert(listbox.size() + 1, "integral_image")

    processedImageFirst = Image.fromarray((transformedImage * 255).astype(np.uint8)).convert("RGB")
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

In [6]:
# Intensity

def intensity():
    global processedImageFirst
    image = np.asarray(processedImageFirst)
    
    intensityValue1 = simpledialog.askstring("Intensity", "Value", parent=root)
    intensityValue2 = simpledialog.askstring("Intensity", "Value", parent=root)
    intensitiedImage = rescale_intensity(image, out_range=(int(intensityValue1), int(intensityValue2)))
    listbox.insert(listbox.size() + 1, "intensity")
    
    processedImageFirst = Image.fromarray((intensitiedImage * 255).astype(np.uint8)).convert("RGB")
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

In [7]:
# Morphology

def morphologies(kind):
    global processedImageFirst
    image = np.asarray(processedImageFirst).astype(np.float32)
    
    if kind == "area_closing":
        morphologiedImage = area_closing(image)
        listbox.insert(listbox.size() + 1, "area_closing")
    if kind == "binary_closing":
        morphologiedImage = binary_closing(image)
        listbox.insert(listbox.size() + 1, "binary_closing")
    if kind == "black_tophat":
        morphologiedImage = black_tophat(image)
        listbox.insert(listbox.size() + 1, "black_tophat")
    if kind == "convex_hull_image":
        morphologiedImage = convex_hull_image(image)
        listbox.insert(listbox.size() + 1, "convex_hull_image")
    if kind == "dilation":
        morphologiedImage = dilation(image)
        listbox.insert(listbox.size() + 1, "dilation")
    if kind == "erosion":
        morphologiedImage = erosion(image)
        listbox.insert(listbox.size() + 1, "erosion")
    if kind == "local_maxima":
        morphologiedImage = local_maxima(image)
        listbox.insert(listbox.size() + 1, "local_maxima")
    if kind == "local_minima":
        morphologiedImage = local_minima(image)
        listbox.insert(listbox.size() + 1, "local_minima")
    if kind == "watershed":
        morphologiedImage = watershed(image)
        listbox.insert(listbox.size() + 1, "watershed")
    if kind == "white_tophat":
        morphologiedImage = white_tophat(image)
        listbox.insert(listbox.size() + 1, "white_tophat")
    
    processedImageFirst = Image.fromarray((morphologiedImage * 255).astype(np.uint8)).convert("RGB")
    processedImage = fitImageToCanvas(processedImageFirst)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

In [8]:
# Active Contour (Segmentation)

def activeContour():
    global processedImageFirst
    image = rgb2gray(np.asarray(processedImageFirst))
    %matplotlib qt
    
    s = np.linspace(0, 2*np.pi, 100)
    r = 100 + 100 * np.sin(s)
    c = 220 + 100 * np.cos(s)
    init = np.array([r, c]).T
    
    snake = active_contour(gaussian(image, 3), init, alpha=0.015, beta=10, gamma=0.001)

    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(image, cmap=plt.cm.gray)
    ax.plot(init[:, 1], init[:, 0], '--r', lw=3)
    ax.plot(snake[:, 1], snake[:, 0], '-b', lw=3)
    ax.set_xticks([]), ax.set_yticks([])
    ax.axis([0, image.shape[1], image.shape[0], 0])

    plt.show()

In [9]:
# Instagram Filter

def putDogFilter(dog, fc, x, y, w, h):
    faceWidth = w
    faceHeight = h
    dog = cv2.resize(dog, (int(faceWidth * 1.5), int(faceHeight * 1.95)))
    for i in range(int(faceHeight * 1.75)):
        for j in range(int(faceWidth * 1.5)):
            for k in range(3):
                if dog[i][j][k] < 235:
                    fc[y + i - int(0.375 * h) - 1][x + j - int(0.35 * w)][k] = dog[i][j][k]
    return fc

def instagramFilter():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    dog = io.imread(os.path.abspath(os.getcwd()) + "\\Samples\\dog.png")
    image = io.imread(filePath)
    
    grayImage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    fl = face_cascade.detectMultiScale(grayImage, 1.09,7)
    
    for (x, y, w, h) in fl: frame = putDogFilter(dog, image, x, y, w, h)
    reference = np.array(Image.open(os.path.abspath(os.getcwd()) + "\\Samples\\second_filter.jpg").convert("RGB"))
    matched = match_histograms(frame, reference, multichannel=True)
    
    global processedImage
    processedImage = Image.fromarray(matched)
    processedImage = fitImageToCanvas(processedImage)
    processedImage = ImageTk.PhotoImage(processedImage)
    
    processedPanel.configure(image = processedImage)
    processedPanel.image = processedImage

In [10]:
# Video İşleme

def faceDetection():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
    
    videoPath = filedialog.askopenfilename(title = "Select an video", filetypes=[("Video Files", (".mp4"))])
    cap = cv2.VideoCapture(videoPath)
    
    while 1:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]

            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

        cv2.imshow("Face and Eye Detection (Press ESC to exit)",img)
        k = cv2.waitKey(30) & 0xff
        if k == 27: break
            
    cap.release()
    cv2.destroyAllWindows()

In [11]:
# Form Root
root = Tk()
root.title("Image Processing Project 1")
root.geometry("1140x415")
root.resizable(False, False)

# Menu
menuBar = Menu(root)

# File Menu
fileMenu = Menu(menuBar, tearoff=0)
fileMenu.add_command(label="Open Image", command=openImageFileDialog)
fileMenu.add_command(label="Save Image", command=saveImage)
menuBar.add_cascade(label="File", menu=fileMenu)

# Filter Menu
filterMenu = Menu(menuBar, tearoff=0)
filterMenu.add_command(label="Hessian", command=lambda:filtersOther("hessian"))
filterMenu.add_command(label="Laplace", command=lambda:filtersOther("laplace"))
filterMenu.add_command(label="Meijering", command=lambda:filtersOther("meijering"))
filterMenu.add_command(label="Roberts", command=lambda:filtersOther("roberts"))
filterMenu.add_command(label="Sato", command=lambda:filtersOther("sato"))
filterMenu.add_command(label="Sobel", command=lambda:filtersOther("sobel"))
filterMenu.add_command(label="Unsharp Mask", command=lambda:filtersOther("unsharp_mask"))
filterMenu.add_command(label="Threshold Li", command=lambda:filtersThreshold("li"))
filterMenu.add_command(label="Threshold Otsu", command=lambda:filtersThreshold("otsu"))
filterMenu.add_command(label="Threshold Yen", command=lambda:filtersThreshold("yen"))
menuBar.add_cascade(label="Filter", menu=filterMenu)

# Histogram Menu
histogramMenu = Menu(menuBar, tearoff=0)
histogramMenu.add_command(label="Histogram Show", command=histogramShowing)
histogramMenu.add_command(label="Histogram Match", command=histogramMatching)
menuBar.add_cascade(label="Histogram", menu=histogramMenu)

# Transformation Menu
transformationMenu = Menu(menuBar, tearoff=0)
transformationMenu.add_command(label="Resize", command=lambda:transformations("resize"))
transformationMenu.add_command(label="Rotate", command=lambda:transformations("rotate"))
transformationMenu.add_command(label="Rescale", command=lambda:transformations("rescale"))
transformationMenu.add_command(label="Swirl", command=lambda:transformations("swirl"))
transformationMenu.add_command(label="Integral Image", command=lambda:transformations("integral_image"))
menuBar.add_cascade(label="Transformation", menu=transformationMenu)

# Intensity Menu
intensityMenu = Menu(menuBar, tearoff=0)
intensityMenu.add_command(label="Rescaling Intensity", command=intensity)
menuBar.add_cascade(label="Intensity", menu=intensityMenu)

# Morphology Menu
morphologyMenu = Menu(menuBar, tearoff=0)
morphologyMenu.add_command(label="Area Closing", command=lambda:morphologies("area_closing"))
morphologyMenu.add_command(label="Binary Closing", command=lambda:morphologies("binary_closing"))
morphologyMenu.add_command(label="Black Tophat", command=lambda:morphologies("black_tophat"))
morphologyMenu.add_command(label="Convex Hull Image", command=lambda:morphologies("convex_hull_image"))
morphologyMenu.add_command(label="Dilation", command=lambda:morphologies("dilation"))
morphologyMenu.add_command(label="Erosion", command=lambda:morphologies("erosion"))
morphologyMenu.add_command(label="Local Maxima", command=lambda:morphologies("local_maxima"))
morphologyMenu.add_command(label="Local Minima", command=lambda:morphologies("local_minima"))
morphologyMenu.add_command(label="Watershed", command=lambda:morphologies("watershed"))
morphologyMenu.add_command(label="White Tophat", command=lambda:morphologies("white_tophat"))
menuBar.add_cascade(label="Morphology", menu=morphologyMenu)

# Active Contour Menu (Segmentation)
activeContourMenu = Menu(menuBar, tearoff=0)
activeContourMenu.add_command(label="Active Contour (Segmentation)", command=activeContour)
menuBar.add_cascade(label="Active Contour", menu=activeContourMenu)

# Instagram Filter Menu
instagramFilterMenu = Menu(menuBar, tearoff=0)
instagramFilterMenu.add_command(label="Instagram Filter", command=instagramFilter)
menuBar.add_cascade(label="Dog Filter", menu=instagramFilterMenu)

# Video Processing Menu
videoProcessingMenu = Menu(menuBar, tearoff=0)
videoProcessingMenu.add_command(label="Video Processing", command=faceDetection)
menuBar.add_cascade(label="Video Processing", menu=videoProcessingMenu)

# Default Image
defaultImage = Image.open(os.path.abspath(os.getcwd()) + "\\Samples\\Blank.png")
defaultImage = fitImageToCanvas(defaultImage)
defaultImage = ImageTk.PhotoImage(defaultImage)

defaultPanel = Label(root, image=defaultImage)
defaultPanel.image = defaultImage
defaultPanel.pack()
defaultPanel.place(x=5, y=5)

# Processed Image
processedImageFirst = Image.open(os.path.abspath(os.getcwd()) + "\\Samples\\Blank.png")
processedImage = fitImageToCanvas(processedImageFirst)
processedImage = ImageTk.PhotoImage(processedImage)

processedPanel = Label(root, image=processedImage)
processedPanel.image = processedImage
processedPanel.pack()
processedPanel.place(x=415, y=5)

# List Box
listbox = Listbox(root, height = 25, width = 50) 
listbox.pack()
listbox.place(x=830, y=5)

In [12]:
# Loop for Screen
root.config(menu=menuBar)
root.mainloop()

C:\Users\Unknown\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The coordinates used by `active_contour` will change from xy coordinates (transposed from image dimensions) to rc coordinates in scikit-image 0.18. Set `coordinates='rc'` to silence this warning. `coordinates='xy'` will restore the old behavior until 0.18, but will stop working thereafter.
  del sys.path[0]
